In [1]:
import math
import numpy as np
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
import rpy2.robjects.lib.ggplot2 as gg
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import IntVector, FloatVector
from rpy2.robjects.lib import grid

from rpy2.robjects import pandas2ri
pandas2ri.activate()

/data/sw1/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/data/sw1/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/data/sw1/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 1: package ‘methods’ was built under R version 3.4.2 

  warnings.warn(x, RRuntimeWarning)
/data/sw1/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 2: package ‘datasets’ was built under R version 3.4.2 

  warnings.warn(x, RRuntimeWarning)
/data/sw1/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 3: package ‘utils’ was built under R version 3.4.2 

  warnings.warn(x, RRuntimeWarning)
/data/sw1/anaconda3/envs/tensorflow3/lib/pyt

In [2]:
grdevices = importr('grDevices')
base = importr('base')

In [3]:
fn = '/data/sw1/Dropbox/timeseries/gen_truth_class_pois.R'
file = ''.join(open(fn,'r').readlines())

In [4]:
mbts = SignatureTranslatedAnonymousPackage(file,'mbts')

/data/sw1/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

  warnings.warn(x, RRuntimeWarning)
/data/sw1/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ✔ tibble  1.3.4     ✔ purrr   0.2.4
✔ tidyr   0.7.2     ✔ dplyr   0.7.4
✔ readr   1.1.1     ✔ stringr 1.2.0
✔ tibble  1.3.4     ✔ forcats 0.2.0

  warnings.warn(x, RRuntimeWarning)
/data/sw1/anaconda3/envs/tensorflow3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

  warnings.warn(x, RRuntimeWarning)


In [5]:
def plot_signal(obj, path, n=4, w=1000, h=1000):
    
    dat_sig = mbts.prep_sig_mbts(obj,6)
    
    pp = gg.ggplot(dat_sig) + \
    gg.aes_string(x='t',y='w') + \
    gg.facet_grid(ro.Formula('sim ~ .')) + \
    gg.geom_line() + \
    gg.theme_classic() 
    
    grdevices.png(file=path, width=w, height=h)
    pp.plot()
    grdevices.dev_off()
    
def plot_simulation(obj, path, i=1, w=1000, h=1000):
    
    dat_sig = mbts.prep_sim_mbts(obj,i)

    pp = gg.ggplot(dat_sig) + \
    gg.geom_rect(gg.aes_string(xmin='min_t',xmax='max_t',ymin=-math.inf,ymax=math.inf),fill='gray') + \
    gg.geom_line(gg.aes_string(x='t',y='signal'),linetype=3,color='red') + \
    gg.geom_line(gg.aes_string(x='t',y='count'),alpha=.5) + \
    gg.geom_point(gg.aes_string(x='t',y='count'),alpha=1) + \
    gg.facet_grid(ro.Formula('taxa ~ .')) + \
    gg.stat_smooth(gg.aes_string(x='t',y='count'),method='loess',color='green',se=False,span=.1,size=.7,alpha=.5) + \
    gg.theme_classic() + \
    gg.labs(x='time',y='count')

    grdevices.png(file=path, width=w, height=h)
    pp.plot()
    grdevices.dev_off()
    
def get_params(obj):
    return dict(zip(obj.names, map(list,list(obj))))

In [6]:
dat = mbts.gen_table(fl_sig=0, # floor of the arima signal normalization
                     w_sig=6, # roof of the arima signal normalization
                     fl_bg=-6, # floor of the background normalization
                     w_bg=6, # roof of the background normalization
                     bg_disp_mu=0, # background noise poisson distribution mean
                     bg_disp_sigma=1, # background noise poisson distribution sd
                     sig_disp_mu1=0, # arima shared noise poisson distribution mean
                     sig_disp_sigma1=0, # arima shared noise poisson distribution sd
                     sig_disp_mu2=0, # signal taxa specific noise poission distribution mean
                     sig_disp_sigma2=1, # signal taxa specific noise poission distribution sd
                     n_sig=10, # number of arima signals
                     n_clust=10, # number of taxa in an arima signal
                     n_tax_sig=1, # number of taxa with FINAL signal (in beta atm)
                     n_bg=700, # number of background taxa (this + signal columns in output data)
                     len_arima=1000, # length of arima signal, needs to be larger than window
                     len_ts=500, # length of the time series (row size of output table)
                     len_signal=300) # length of the shared signal

In [8]:
dat_table = np.asarray(dat)

In [13]:
get_params(dat.slots['params'])

{'bg_disp_mu': [0],
 'bg_disp_sigma': [1],
 'fl_bg': [-6],
 'fl_sig': [0],
 'len_arima': [1000],
 'len_signal': [300],
 'len_ts': [500],
 'n_bg': [700],
 'n_clust': [10],
 'n_sig': [10],
 'n_tax_sig': [1],
 's_sig': [2],
 'sig_disp_mu1': [0],
 'sig_disp_mu2': [0],
 'sig_disp_sigma1': [0],
 'sig_disp_sigma2': [0.5],
 'w_bg': [6],
 'w_sig': [6]}

In [15]:
mbts.sparsity_mbts(dat)

    signal 
background 


 0.0066200 
 0.3231171 




R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f63cf268e48 / R:0x74a8fd0>
[0.006620, 0.323117]

In [16]:
mbts.quantiles_mbts(dat)

         
 signal
 background

75%      
     42
          5

76.78571%
     47
          5

78.57143%
     52
          6

80.35714%
     56
          6

82.14286%
     62
          7

83.92857%
     69
          8

85.71429%
     75
          9

87.5%    
     84
         10

89.28571%
     94
         11

91.07143%
    109
         14

92.85714%
    127
         16

94.64286%
    149
         21

96.42857%
    190
         28

98.21429%
    281
         45

100%     
    444
       3180




R object with classes: ('matrix',) mapped to:
<Matrix - Python:0x7f63cf269e48 / R:0x708df40>
[42.000000, 47.000000, 52.000000, ..., 28.000000, 45.000000, 3180.000000]

In [20]:
np.asarray(mbts.sig_cor_mbts(dat,i=1))

array([[ 1.   ,  0.988,  0.986,  0.986,  0.988,  0.988,  0.988,  0.987,
         0.987,  0.989],
       [ 0.988,  1.   ,  0.989,  0.988,  0.992,  0.989,  0.99 ,  0.99 ,
         0.99 ,  0.988],
       [ 0.986,  0.989,  1.   ,  0.988,  0.989,  0.987,  0.989,  0.988,
         0.987,  0.988],
       [ 0.986,  0.988,  0.988,  1.   ,  0.989,  0.988,  0.988,  0.989,
         0.988,  0.988],
       [ 0.988,  0.992,  0.989,  0.989,  1.   ,  0.991,  0.991,  0.991,
         0.99 ,  0.99 ],
       [ 0.988,  0.989,  0.987,  0.988,  0.991,  1.   ,  0.989,  0.988,
         0.99 ,  0.989],
       [ 0.988,  0.99 ,  0.989,  0.988,  0.991,  0.989,  1.   ,  0.99 ,
         0.99 ,  0.99 ],
       [ 0.987,  0.99 ,  0.988,  0.989,  0.991,  0.988,  0.99 ,  1.   ,
         0.989,  0.989],
       [ 0.987,  0.99 ,  0.987,  0.988,  0.99 ,  0.99 ,  0.99 ,  0.989,
         1.   ,  0.988],
       [ 0.989,  0.988,  0.988,  0.988,  0.99 ,  0.989,  0.99 ,  0.989,
         0.988,  1.   ]])

In [21]:
plot_signal(dat,path='/data/sw1/Dropbox/timeseries/sig.png')
plot_simulation(dat,i=1,path='/data/sw1/Dropbox/timeseries/sim.png')